In [2]:
#SHAP
import shap
import keras.models
from keras.layers import Dense, Dropout, Input, Conv1D, Concatenate, Flatten
import numpy as np
import tensorflow as tf
import pandas as pd

In [ ]:
# def mlp_model(input_dim, width, depth, dropout):
#     """ Function to create the MLP model. """
#     model = keras.models.Sequential()

#     for i in range(0, depth):
#         model.add(Dense(units=width, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
#         model.add(Dropout(dropout))

#     model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return model

In [ ]:
# def mlp_model_reweight(input_dim, width, depth, dropout, weight_vector):
#     """ Function to create the MLP model with reweighting. """
#     model = keras.models.Sequential()

#     for i in range(0, depth):
#         model.add(Dense(units=width, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
#         model.add(Dropout(dropout))

#     model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))


#     loss = reweighted_binary_crossentropy(weight_vector)
#     model.compile(loss=loss, optimizer='adam', metrics=['accuracy'])

#     return model

In [3]:
def load_selected_model(folder_name):
    
    with open(folder_name + '/best_epoch_number.txt', 'r') as file:
        best_epo = int(file.read())
    
    model = tf.keras.models.load_model(folder_name + '/checkpoint/cp-{epoch:04d}.ckpt'.format(epoch=best_epo))
    #model.load_weights(folder_name + '/checkpoint/cp-{epoch:04d}.ckpt'.format(epoch=best_epo) + '/variables')
    
    return model

In [4]:
def load_selected_model_dp(folder_name, group_name):        
        
        
    with open(folder_name + '/best_dp_unit_number_whole_group_threshold.txt', 'r') as file:
        best_unit = file.read().split(' ')[0] # number only

    if best_unit == '1':
        print('best unit is 1 (original)')
        return 

    else:
        folder_name += '/' + folder_name.split('/')[-2] + '-' + best_unit + 'u'

        with open(folder_name + '/best_epoch_number_whole_group_thre_' + group_name + '.txt', 'r') as file:
            best_epo = file.read()
    
    model = tf.keras.models.load_model(folder_name + '/checkpoint/cp-{epoch:04d}.ckpt'.format(epoch=int(best_epo)))
    
    return model

In [5]:
def load_selected_model_dp_reweight(folder_name, group_name):        
        
        
    with open(folder_name + '/best_dp_unit_number_whole_group_threshold.txt', 'r') as file:
        best_unit = file.read().split(' ')[0] # number only

    if best_unit == '1':
        print('best unit is 1 (original)')
        return 

    else:
        folder_name += '/' + folder_name.split('/')[-2] + '-' + best_unit + 'u'

        with open(folder_name + '/best_epoch_number_whole_group_thre_' + group_name + '.txt', 'r') as file:
            best_epo = file.read()
    
    model = tf.keras.models.load_model(folder_name + '/checkpoint/cp-{epoch:04d}.ckpt'.format(epoch=int(best_epo)))
    
    return model

In [6]:
def load_data(cancer):
    if cancer == 'breast':
        X_train = np.load('../experiments/data-bcs/X_train_normalized.npy')
        X_test = np.load('../experiments/data-bcs/X_test_normalized.npy')
    else:
        X_train = np.load('../experiments/data-lcs/X_train_normalized.npy')
        X_test = np.load('../experiments/data-lcs/X_test_normalized.npy')
    
    np.random.seed(0)
    X_train_100 = X_train[np.random.choice(X_train.shape[0], 100)]
    X_test_100 = X_test[np.random.choice(X_test.shape[0], 100)]
    
    return X_train_100, X_test_100

In [7]:
def read_columns(filename):
    with open(filename, 'r') as f:
        lines = f.readlines();
        #print(len(lines))
        variables = []
        variable_names = []
        previous = ''
        count = 0
        for i in range(len(lines)):
            l = lines[i]
#             if 'continuous' in l:
#                 variables.append((i, 1))
#                 #print(l)
#                 #count = 1
#             else:
            current = " ".join(l.split(" ")[:-1]) # remove the number at the end
            if previous == current: # same variable
                count += 1
            else: # previous != current including previous == '', new variable
                # append the count for previous 
                if previous != '':
                    variables.append((i-count, count))
                    variable_names.append(previous)
                # start new
                count = 1
            previous = current

        variables.sort(key = lambda x: x[0]) 
        #variables = np.array(variables).astype(int)

    #return np.array(lines)[variables[:0]], variables
    return variable_names, variables

In [8]:
def calculate_shap_values(model, background, test):
    
    print("data loaded")
    explainer = shap.KernelExplainer(model, background)
    print("explainer created")
    shap_values = explainer.shap_values(test)
    print("shap values calculated")
    
    shap_values = np.array(shap_values)
    shap_values = shap_values.reshape(test.shape) # 3 dimentional, only need (sample, feature) 2d array
    
    shap_values_mean = np.mean(np.abs(shap_values), axis=0)
    
    return shap_values_mean

In [9]:
def feature_importance_to_file(folder_name, col_filename, shap_values_mean):
    
    feature_name, feature_index = read_columns(col_filename)
    
    # sum importance
    sum_values = []
    for i in feature_index:
        sum_values.append(np.sum(shap_values_mean[i[0]:i[0]+i[1]]))
        
    sum_sorted_index = np.argsort(sum_values)[::-1]
    
    sum_data = np.array([np.array(feature_name)[sum_sorted_index], np.array(sum_values)[sum_sorted_index]])
    
    df_sum = pd.DataFrame(data=np.transpose(sum_data),
                         columns=['Feature', 'Importance'], )
    
    df_sum.to_csv(folder_name + '/SHAP_feature_importance_sum.csv', index=False)
    
    # average importance
    mean_values = []
    for i in feature_index:
        mean_values.append(np.mean(shap_values_mean[i[0]:i[0]+i[1]]))
        
    mean_sorted_index = np.argsort(mean_values)[::-1]
    
    mean_data = np.array([np.array(feature_name)[mean_sorted_index], np.array(mean_values)[mean_sorted_index]])
    
    df_mean = pd.DataFrame(data=np.transpose(mean_data),
                         columns=['Feature', 'Importance'], )
    
    df_mean.to_csv(folder_name + '/SHAP_feature_importance_mean.csv', index=False)

In [10]:
def generate_importance(folder_name, col_filename, cancer):

    model = load_selected_model(folder_name)
    X_train_100, X_test = load_data(cancer)
    
    shap_values = calculate_shap_values(model, X_train_100, X_test)
    
    feature_importance_to_file(folder_name, col_filename, shap_values)

In [11]:
def generate_importance_dp(folder_name, col_filename, cancer, group_name):

    model = load_selected_model_dp(folder_name, group_name)
    X_train_100, X_test = load_data(cancer)
    
    shap_values = calculate_shap_values(model, X_train_100, X_test)
    
    feature_importance_to_file(folder_name, col_filename, shap_values)

In [ ]:
######## debug and run

In [10]:
# bcs original
generate_importance('../experiments/bcs-original/bcs-original-2', '../experiments/bcs_input_columns.txt', 'breast')

data loaded
explainer created


  0%|          | 0/100 [00:00<?, ?it/s]

shap values calculated


In [11]:
# lcs original
generate_importance('../experiments/lcs-original/lcs-original-2', '../experiments/lcs_input_columns.txt', 'lung')

data loaded
explainer created


  0%|          | 0/100 [00:00<?, ?it/s]

shap values calculated


In [12]:
# bcs dp
generate_importance_dp('../experiments/bcs-dp-asian/bcs-dp-asian-2', '../experiments/bcs_input_columns.txt', 'breast', 'Ethnicity_Asian')
generate_importance_dp('../experiments/bcs-dp-4050/bcs-dp-4050-2', '../experiments/bcs_input_columns.txt', 'breast', '40<=Age<50')

data loaded
explainer created


  0%|          | 0/100 [00:00<?, ?it/s]

shap values calculated
data loaded
explainer created


  0%|          | 0/100 [00:00<?, ?it/s]

shap values calculated


In [13]:
# lcs dp
generate_importance_dp('../experiments/lcs-dp-black/lcs-dp-black-2', '../experiments/lcs_input_columns.txt', 'lung', 'Ethnicity_Black')
generate_importance_dp('../experiments/lcs-dp-age90/lcs-dp-age90-2', '../experiments/lcs_input_columns.txt', 'lung', 'Age>=90')

data loaded
explainer created


  0%|          | 0/100 [00:00<?, ?it/s]

shap values calculated
data loaded
explainer created


  0%|          | 0/100 [00:00<?, ?it/s]

shap values calculated


In [12]:
generate_importance_dp('../experiments/lcs-dp-black/lcs-dp-black-3', '../experiments/lcs_input_columns.txt', 'lung', 'Ethnicity_Black')
generate_importance_dp('../experiments/lcs-dp-age90/lcs-dp-age90-3', '../experiments/lcs_input_columns.txt', 'lung', 'Age>=90')

data loaded
explainer created


  0%|          | 0/100 [00:00<?, ?it/s]

shap values calculated
data loaded
explainer created


  0%|          | 0/100 [00:00<?, ?it/s]

shap values calculated


In [13]:
generate_importance_dp('../experiments/lcs-dp-8090/lcs-dp-8090-2', '../experiments/lcs_input_columns.txt', 'lung', '80<=Age<90')

data loaded
explainer created


  0%|          | 0/100 [00:00<?, ?it/s]

shap values calculated


In [14]:
# bcs reweight
generate_importance('../experiments/bcs-reweight/bcs-reweight-2', '../experiments/bcs_input_columns.txt', 'breast')

data loaded
explainer created


  0%|          | 0/100 [00:00<?, ?it/s]

shap values calculated


In [15]:
# lcs reweight
generate_importance('../experiments/lcs-reweight/lcs-reweight-2', '../experiments/lcs_input_columns.txt', 'lung')

data loaded
explainer created


  0%|          | 0/100 [00:00<?, ?it/s]

shap values calculated


In [16]:
# bcs dp reweight
generate_importance_dp('../experiments/bcs-dpreweight-asian/bcs-dpreweight-asian-1', '../experiments/bcs_input_columns.txt', 'breast', 'Ethnicity_Asian')
generate_importance_dp('../experiments/bcs-dpreweight-4050/bcs-dpreweight-4050-1', '../experiments/bcs_input_columns.txt', 'breast', '40<=Age<50')

data loaded
explainer created


  0%|          | 0/100 [00:00<?, ?it/s]

shap values calculated
data loaded
explainer created


  0%|          | 0/100 [00:00<?, ?it/s]

shap values calculated


In [25]:
### debug
filename = '../experiments/bcs_input_columns.txt'
feature_name, feature_index = read_columns(filename)

In [5]:
shap_values = calculate_shap_values('breast', '../experiments/bcs-original/bcs-original-2')

data loaded
explainer created


  0%|          | 0/1000 [00:00<?, ?it/s]

shap values calculated


In [6]:
len(shap_values[0])

1000

In [7]:
len(shap_values[0][0])

1418

In [10]:
np.save('../experiments/bcs-original/bcs-original-2/shap_values_1000.npy', np.array(shap_values))

In [90]:
shao_vales = np.load('../experiments/bcs-original/bcs-original-2/shap_values_1000.npy')

In [91]:
shap_values = np.array(shap_values)

In [92]:
shap_values.shape

(100, 1418)

In [20]:
shap_values_mean = np.mean(np.abs(shap_values), axis=0)

In [21]:
shap_values_mean.shape

(1418,)

In [22]:
shap_values_mean[:20]

array([9.59678878e-03, 9.98602008e-03, 1.61895978e-03, 1.91449599e-02,
       2.66662141e-04, 1.50743645e-04, 1.10340761e-02, 7.25920761e-04,
       1.34909806e-02, 0.00000000e+00, 5.10857126e-04, 0.00000000e+00,
       1.29582616e-04, 4.10512926e-05, 5.96067829e-04, 1.80015142e-02,
       0.00000000e+00, 1.97619834e-04, 0.00000000e+00, 8.07333686e-05])

In [26]:
merged_values = []
for i in feature_index:
    merged_values.append(np.sum(shap_values_mean[i[0]:i[0]+i[1]]))

In [27]:
len(merged_values)

56

In [28]:
len(feature_name)

56

In [31]:
sorted_index = np.argsort(merged_values)[::-1]

In [32]:
np.array(feature_name)[sorted_index]

array(['State-county recode', 'EOD 10 - positive lymph nodes examined',
       'Grade', 'CS Tumor size', 'CS Site-Specific Factor 3',
       'EOD 10 - number of lymph nodes examined', 'SEER registry',
       'Derived AJCC T', 'Adjusted AJCC 6th T (1988+)', 'Year of birth',
       'Age at diagnosis', 'Derived AJCC N', 'Historic SSG 2000 Stage',
       'Adjusted AJCC 6th Stage (1988+)', 'Recode ICD-O-2 to 9',
       'Histologic Type ICD-O-2', 'Derived AJCC Stage Group',
       'Histologic Type ICD-O-3', 'Derived SS1977',
       'CS Site-Specific Factor 1', 'Derived SS2000',
       'SEER historic stage A', 'ER Status Recode Breast Cancer (1990+)',
       'Insurance Recode (2007+)', 'CS Lymph Nodes',
       'CS Site-Specific Factor 4', 'CS Site-Specific Factor 2',
       'Month of diagnosis', 'Marital status at diagnosis',
       'PR Status Recode Breast Cancer (1990+)',
       'CS Version Input Original', 'Adjusted AJCC 6th M (1988+)',
       'Race/ethnicity', 'CS Site-Specific Factor 5',

In [33]:
merged_values2 = []
for i in feature_index:
    merged_values2.append(np.mean(shap_values_mean[i[0]:i[0]+i[1]]))

In [34]:
sorted_index2 = np.argsort(merged_values2)[::-1]

In [35]:
np.array(feature_name)[sorted_index2]

array(['Year of birth', 'Age at diagnosis',
       'EOD 10 - positive lymph nodes examined', 'Grade',
       'EOD 10 - number of lymph nodes examined',
       'Age recode <1 year olds', 'Historic SSG 2000 Stage',
       'CS Tumor size', 'Year of diagnosis', 'SEER historic stage A',
       'ER Status Recode Breast Cancer (1990+)', 'Derived SS1977',
       'Derived SS2000', 'PR Status Recode Breast Cancer (1990+)',
       'CS Site-Specific Factor 1', 'CS Site-Specific Factor 4',
       'Insurance Recode (2007+)', 'Derived AJCC T',
       'Adjusted AJCC 6th M (1988+)', 'SEER registry',
       'Adjusted AJCC 6th Stage (1988+)', 'Adjusted AJCC 6th T (1988+)',
       'CS Site-Specific Factor 2', 'Derived AJCC Stage Group',
       'Marital status at diagnosis', 'CS Site-Specific Factor 5',
       'Origin Recode NHIA', 'CS Version Input Current', 'Derived AJCC N',
       'Recode ICD-O-2 to 9', 'Derived AJCC M', 'Month of diagnosis',
       'Race recode A', 'CS Site-Specific Factor 6', 'CS Mets

In [38]:
# test difference between using 100 samples and 1000 samples
# using only 100 test samples
# much faster
shap_values = calculate_shap_values('breast', '../experiments/bcs-original/bcs-original-2')[0]

data loaded
explainer created


  0%|          | 0/100 [00:00<?, ?it/s]

shap values calculated


In [47]:
shap_values_mean = np.mean(np.abs(shap_values)[0], axis=0)

In [48]:
shap_values_mean.shape

(1418,)

In [49]:
merged_values = []
for i in feature_index:
    merged_values.append(np.sum(shap_values_mean[i[0]:i[0]+i[1]]))

In [50]:
sorted_index = np.argsort(merged_values)[::-1]
np.array(feature_name)[sorted_index]

array(['State-county recode', 'EOD 10 - positive lymph nodes examined',
       'CS Tumor size', 'Grade', 'CS Site-Specific Factor 3',
       'EOD 10 - number of lymph nodes examined', 'Derived AJCC T',
       'SEER registry', 'Adjusted AJCC 6th T (1988+)', 'Derived AJCC N',
       'Age at diagnosis', 'Year of birth',
       'Adjusted AJCC 6th Stage (1988+)', 'Historic SSG 2000 Stage',
       'Recode ICD-O-2 to 9', 'Derived AJCC Stage Group',
       'Histologic Type ICD-O-2', 'Histologic Type ICD-O-3',
       'CS Site-Specific Factor 1', 'Derived SS1977', 'Derived SS2000',
       'ER Status Recode Breast Cancer (1990+)', 'SEER historic stage A',
       'CS Lymph Nodes', 'Insurance Recode (2007+)',
       'Marital status at diagnosis', 'CS Site-Specific Factor 4',
       'CS Site-Specific Factor 2', 'Month of diagnosis',
       'CS Version Input Original',
       'PR Status Recode Breast Cancer (1990+)', 'Race/ethnicity',
       'CS Site-Specific Factor 6', 'Adjusted AJCC 6th M (1988+)',

In [ ]:
merged_values2 = []
for i in feature_index:
    #print(i)
    #print(shap_values_mean[i[0]:i[0]+i[1]])
    merged_values2.append(np.mean(shap_values_mean[i[0]:i[0]+i[1]]))

In [52]:
sorted_index2 = np.argsort(merged_values2)[::-1]
np.array(feature_name)[sorted_index2]

array(['Age at diagnosis', 'Year of birth',
       'EOD 10 - positive lymph nodes examined', 'Grade',
       'EOD 10 - number of lymph nodes examined',
       'Historic SSG 2000 Stage', 'CS Tumor size',
       'Age recode <1 year olds',
       'ER Status Recode Breast Cancer (1990+)', 'Year of diagnosis',
       'SEER historic stage A', 'Derived SS1977', 'Derived SS2000',
       'CS Site-Specific Factor 4', 'CS Site-Specific Factor 1',
       'PR Status Recode Breast Cancer (1990+)',
       'Insurance Recode (2007+)', 'Derived AJCC T',
       'Marital status at diagnosis', 'SEER registry',
       'Adjusted AJCC 6th Stage (1988+)', 'Adjusted AJCC 6th T (1988+)',
       'Adjusted AJCC 6th M (1988+)', 'Derived AJCC Stage Group',
       'CS Site-Specific Factor 5', 'CS Site-Specific Factor 2',
       'Derived AJCC N', 'Race recode A', 'Type of reporting source',
       'Recode ICD-O-2 to 9', 'CS Site-Specific Factor 6',
       'Derived AJCC M', 'Month of diagnosis', 'CS Version Input Curre

In [57]:
np.array(merged_values2)[sorted_index2]

array([9.42595707e-03, 9.03502303e-03, 6.97935795e-03, 4.74283720e-03,
       2.36299495e-03, 2.04961309e-03, 1.86677071e-03, 1.85209400e-03,
       1.56083309e-03, 1.52983179e-03, 1.52572651e-03, 1.17412265e-03,
       1.13868212e-03, 1.01285032e-03, 1.01240789e-03, 9.68949048e-04,
       9.05275645e-04, 8.49123500e-04, 7.60363204e-04, 7.51704565e-04,
       7.47676941e-04, 7.20892535e-04, 6.95032880e-04, 6.78525126e-04,
       6.66718098e-04, 6.58981059e-04, 4.73109997e-04, 4.44510320e-04,
       4.41093774e-04, 4.25051805e-04, 3.87176415e-04, 3.59396513e-04,
       3.58035047e-04, 3.56723560e-04, 3.20371128e-04, 2.63412597e-04,
       2.57473623e-04, 2.48571613e-04, 2.31875634e-04, 2.30875532e-04,
       1.62312388e-04, 1.43310817e-04, 1.18853711e-04, 8.52497051e-05,
       6.83040652e-05, 6.48976016e-05, 5.88269631e-05, 5.58503689e-05,
       5.37541335e-05, 3.35759736e-05, 6.00677710e-06, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [43]:
sum(shap_values[0])

array([ 0.14054207,  0.12365059, -0.06090906, ..., -0.03224274,
        0.        ,  0.01971833])

In [37]:
model = load_selected_model('../experiments/bcs-original/bcs-original-2')
X_test = np.load('../experiments/data-bcs/X_test_normalized.npy')

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                28380     
_________________________________________________________________
dropout (Dropout)            (None, 20)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                420       
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 28,821
Trainable params: 28,821
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.predict(X_test[100:200])

array([[0.9631458 ],
       [0.9709036 ],
       [0.9916557 ],
       [0.9958331 ],
       [0.99936086],
       [0.6076393 ],
       [0.86075497],
       [0.8749577 ],
       [0.91563714],
       [0.99698657],
       [0.9549452 ],
       [0.79491955],
       [0.96708804],
       [0.6817847 ],
       [0.9787516 ],
       [0.9947973 ],
       [0.9974389 ],
       [0.9080742 ],
       [0.8800338 ],
       [0.9676145 ],
       [0.982733  ],
       [0.9645707 ],
       [0.94979453],
       [0.95160455],
       [0.9973189 ],
       [0.60796523],
       [0.79957056],
       [0.34953058],
       [0.9942006 ],
       [0.96433914],
       [0.8669462 ],
       [0.9941037 ],
       [0.96265846],
       [0.6630537 ],
       [0.9327435 ],
       [0.8435414 ],
       [0.9224866 ],
       [0.10364173],
       [0.98194   ],
       [0.9150522 ],
       [0.9960603 ],
       [0.8547274 ],
       [0.9877537 ],
       [0.8417863 ],
       [0.9993305 ],
       [0.97906953],
       [0.9935514 ],
       [0.982

In [38]:
X_test[0]

array([-0.46882305,  0.46435207, -0.31052175, ...,  0.        ,
        0.        ,  0.        ], dtype=float32)

In [113]:
b = np.array([0.2, 0.3, 0.5])
w = np.array([1, 3, 1])

In [114]:
w * b

ValueError: operands could not be broadcast together with shapes (3,) (4,) 